In [1]:
from tqdm import tqdm
import pandas as pd
import re
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
import torch
from transformers import AutoTokenizer, AutoModel
from soynlp.normalizer import repeat_normalize
import numpy as np
import nltk
from kss import split_sentences

c:\python-capstone\tag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Check for GPU
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
def remove_emoji(text):
    # 이모지를 정규 표현식으로 제거
    emoji_pattern = re.compile(
        "[\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & shapes
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F700-\U0001F77F"  # alchemical symbols
        "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        "\U0001FA00-\U0001FA6F"  # Chess Symbols
        "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        "\U00002702-\U000027B0"  # Dingbats
        "]+",
        flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', text)  # 이모지 제거 후 반환

In [4]:
# Step 1: 데이터 로드
file_path = "C:/Users/cho03/Desktop/대학/4학년/2학기/캡스톤디자인-여행사이트/data/lodging_reviews.csv"
data = pd.read_csv(file_path)

# Step 2: 'review_text' 또는 'rating' 컬럼에 NaN이 있는 행 삭제
data = data.dropna(subset=["review_text", "rating"])

# 전처리 함수 정의
pattern = re.compile(f"[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣]+")
url_pattern = re.compile(
    r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)"
)


def clean(x):
    x = pattern.sub(" ", x)
    x = remove_emoji(x)  # emoji 삭제
    x = url_pattern.sub("", x)
    x = x.strip()
    x = repeat_normalize(x, num_repeats=2)
    
    return x


# Step 3: 리뷰 텍스트 전처리
data["review_after"] = data["review_text"].apply(clean)

# Step 4: 리뷰의 길이를 계산하여 'length' 컬럼에 저장
data["length"] = data["review_after"].str.len()

# Step 5: 리뷰의 길이가 10 미만인 데이터 제거
data = data[data["length"] >= 10]

# Step 6: 각 숙소(lodging_id)별 전체 리뷰를 기준으로 평균 평점과 리뷰 개수 계산
lodging_summary = (
    data.groupby("lodging_id")
    .agg(total_review_count=("rating", "count"), average_rating=("rating", "mean"))
    .reset_index()
)

# 조건 1: 전체 리뷰의 평균 평점이 4 이상인 숙소
qualified_lodging_ids = lodging_summary[lodging_summary["average_rating"] >= 4]["lodging_id"]

# Step 7: 조건을 만족하는 숙소 중 평점이 4 이상인 리뷰만 필터링
good_review = data[(data["lodging_id"].isin(qualified_lodging_ids)) & (data["rating"] >= 4)]

# Step 8: 평점이 4 이상인 리뷰 개수가 5개 이상인 숙소만 선택
final_lodging_summary = good_review.groupby("lodging_id").agg(good_review_count=("rating", "count")).reset_index()
final_lodging_ids = final_lodging_summary[final_lodging_summary["good_review_count"] >= 5]["lodging_id"]

# Step 9: 최종 필터링 - 전체 조건을 만족하는 숙소의 평점 4 이상인 리뷰만 추출
filtered_data = good_review[good_review["lodging_id"].isin(final_lodging_ids)]

In [5]:
tag_examples = {
    "가성비": [
        "가격 대비 만족도가 정말 높았어요.",
        "비용 대비 이 정도 서비스는 예상치 못했어요.",
        "이 가격에 이런 품질이라니 기대 이상입니다.",
        "가격이 저렴하면서도 필요한 편의시설이 다 갖춰져 있어 좋았습니다.",
        "가성비 면에서 다른 곳과 비교할 수 없을 정도로 좋았어요.",
        "저렴한 가격에도 불구하고 서비스가 아주 훌륭했습니다.",
        "가격에 비해 숙소의 상태와 서비스가 만족스러웠습니다.",
        "비용을 생각하면 매우 합리적인 선택이었습니다.",
        "가격이 저렴한데도 시설이 깨끗하고 관리가 잘 되어 있었습니다.",
        "이 정도 퀄리티에 이 가격이면 누구에게나 추천하고 싶어요.",
        "가격에 비해 훨씬 좋은 숙소였습니다.",
        "합리적인 가격에 만족스러운 서비스를 경험했어요.",
        "저렴한 가격이었지만, 기대 이상으로 즐거운 숙박이었어요.",
        "비용 대비 충분히 좋은 선택이라고 생각해요.",
        "경제적인 가격에 시설도 무척 만족스러웠습니다.",
    ],
    "청결": [
        "객실 청결 상태가 매우 잘 유지되고 있었습니다.",
        "방이 굉장히 깔끔하고 정돈된 느낌이었어요.",
        "숙소 전체가 위생적으로 관리되어서 편안하게 머물렀습니다.",
        "화장실과 침대가 모두 매우 청결하게 유지되어 있었어요.",
        "깨끗하게 관리된 공간이라서 마음 편히 지낼 수 있었습니다.",
        "숙소가 정돈이 잘 되어 있고 위생 관리가 철저해 보였습니다.",
        "전체적인 청결 상태가 아주 훌륭했어요.",
        "방 안이 쾌적하고 먼지 하나 없는 느낌이어서 좋았어요.",
        "위생 상태가 매우 잘 유지되고 있어 안심이 되었습니다.",
        "객실이 깨끗하게 관리되어 기분이 좋았습니다.",
        "침구와 바닥 모두 청결해서 불편함이 없었어요.",
        "위생적으로 관리된 숙소라서 마음이 편안했습니다.",
        "청결도가 높아 누구에게나 추천하고 싶은 숙소예요.",
        "정리 정돈이 잘 되어 있어 아주 쾌적했습니다.",
        "위생 관리가 철저히 이루어져서 신뢰가 갔습니다.",
    ],
    "직원 만족": [
        "직원들이 정말 친절해서 기분이 좋았습니다.",
        "사장님께서 직접 챙겨주셔서 감사했습니다.",
        "서비스가 아주 세심하고 친절했어요.",
        "프론트 직원들이 친절하게 응대해 주셨어요.",
        "직원들과 사장님 모두 친절하고 따뜻했습니다.",
        "요청 사항에 대해 직원들이 빠르게 응대해 주셨어요.",
        "사장님이 친절하게 맞아주셔서 마음이 편안했어요.",
        "직원들과 사장님 모두 서비스가 훌륭합니다.",
        "친절한 직원들과 사장님 덕분에 편안하게 머물렀습니다.",
        "직원들과 사장님이 정말 배려심 깊게 대해 주셨어요.",
        "사장님이 따뜻하게 대해주셔서 인상적이었어요.",
        "직원들이 협조적이고 사장님도 친절해서 좋았습니다.",
        "서비스가 뛰어나고 직원들이 상냥했어요.",
        "사장님과 직원들 모두 친절하게 맞아주셔서 기분 좋았습니다.",
        "요청에 빠르게 대응해 주셔서 감사했어요.",
    ],
    "위치": [
        "위치가 좋아서 근처 관광지로 이동하기 편리했어요.",
        "도심 한가운데 위치해 있어서 접근성이 뛰어났습니다.",
        "주변에 식당과 카페가 많아 편리했습니다.",
        "대중교통과 가까운 위치라 이동이 매우 편리했어요.",
        "관광 명소들과 가까워서 시간을 절약할 수 있었습니다.",
        "주요 시설과의 접근성이 매우 좋았어요.",
        "숙소의 위치가 아주 좋고 주변에 필요한 것이 모두 있었습니다.",
        "위치가 좋아 어디든 쉽게 갈 수 있었습니다.",
        "주변에 다양한 볼거리와 즐길 거리가 있어 만족스러웠어요.",
        "숙소가 조용한 곳에 위치해 있어서 편히 쉴 수 있었습니다.",
        "대중교통을 이용하기에 최적의 위치에 있습니다.",
        "도심과 가까워 모든 것이 쉽게 접근 가능했습니다.",
        "주요 관광지와 가까운 위치 덕분에 여행이 편리했어요.",
        "교통이 편리하고 중심지와 가까워 편리했습니다.",
        "편리한 위치 덕분에 모든 계획이 순조로웠어요.",
    ],
    "가족 여행": [
        "아이들과 함께 머물기에 좋은 숙소예요.",
        "가족 단위로 오기 딱 좋아요.",
        "가족과 함께 머물기에 필요한 시설이 잘 갖춰져 있어요.",
        "부모님을 모시고 오기 좋은 숙소입니다.",
        "아이들과 부모님 모두 만족할 만한 숙소였어요.",
        "부모님과 함께 이용하기 편리했습니다.",
        "가족이 함께 즐길 수 있는 숙소입니다.",
        "아이들과 부모님이 편안하게 지낼 수 있었어요.",
        "부모님과의 가족 여행에 딱 맞는 숙소입니다.",
        "아이들이 좋아할 편의시설도 잘 갖춰져 있어요.",
        "부모님과 아이들 모두 만족한 여행이었어요.",
        "가족끼리 머물기에 완벽한 공간입니다.",
        "가족 여행에 필요한 모든 것을 갖춘 숙소입니다.",
        "부모님과 아이들 모두 즐길 수 있는 숙소예요.",
        "가족이 모두 함께 편안하게 머물 수 있었습니다.",
    ],
    "연인": [
        "로맨틱한 분위기로 커플 여행에 좋습니다.",
        "남자친구와 함께 오기 딱 좋아요.",
        "특별한 날을 위해 여자친구와 방문했어요.",
        "아늑한 분위기라 남자친구와 추천합니다.",
        "연인과 오붓한 시간을 보내기 좋은 숙소입니다.",
        "여자친구와 로맨틱한 시간을 보냈어요.",
        "연인과 함께하는 특별한 장소로 추천합니다.",
        "남자친구와 함께할 아늑한 분위기입니다.",
        "기념일에 여자친구와 방문하기 딱 좋아요.",
        "남자친구와 특별한 날을 보내기 좋았어요.",
        "연인과 오붓하게 머물기 좋은 장소입니다.",
        "여자친구와 머물기에 완벽한 숙소였습니다.",
        "로맨틱한 분위기가 남자친구와 잘 맞았어요.",
        "연인과 함께하기 좋은 로맨틱한 장소입니다.",
        "커플에게 추천하는 멋진 숙소예요.",
    ],
    "풍경": [
        "자연 경관이 아름다워서 힐링되는 느낌이었습니다.",
        "숙소에서 바라보는 뷰가 정말 멋졌어요.",
        "산과 강의 경치가 인상적이고 마음이 편안해졌습니다.",
        "탁 트인 전망 덕분에 기분이 상쾌해졌어요.",
        "자연 속에서 여유를 만끽할 수 있는 숙소였어요.",
        "뷰가 너무 아름다워서 계속 바라보고 싶었습니다.",
        "숙소에서 보이는 자연 경관이 너무 좋아서 기억에 남아요.",
        "풍경이 멋져서 정말 힐링이 되었습니다.",
        "아름다운 자연과 함께하는 느낌이 들어 좋았어요.",
        "숙소에서 보는 전망이 정말 일품이었습니다.",
        "자연의 아름다움을 느끼며 여유로운 시간을 보낼 수 있었습니다.",
        "탁 트인 뷰 덕분에 아침마다 기분이 상쾌했어요.",
        "숙소 주변의 자연 경관이 너무 아름다워서 힐링이 되었습니다.",
        "창밖으로 보이는 풍경이 정말 감동적이었어요.",
        "자연 속에서 조용히 쉴 수 있는 멋진 숙소였습니다.",
    ],
}

# 가중치를 부여할 단어 리스트 정의
tag_keywords = {
    "가성비": ["가격", "저렴", "만족", "가성비", "비용"],
    "청결": ["청결", "깨끗", "위생", "관리", "정돈"],
    "직원 만족": ["친절", "서비스", "직원", "사장님"],
    "위치": ["위치", "접근성", "근처", "교통"],
    "가족 여행": ["가족", "아이", "부모님"],
    "연인": ["연인", "커플", "남자친구", "여자친구", "기념일"],
    "풍경": ["경관", "뷰", "자연", "전망", "풍경"],
}

In [6]:
# KCBERT 모델 및 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-large")
model = AutoModel.from_pretrained("beomi/kcbert-large").to(device)

In [7]:
def get_kcbert_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    outputs = model(**inputs)

    # Use the hidden state of the [CLS] token for sentence representation
    cls_embedding = outputs.last_hidden_state[:, 0, :]  # Get [CLS] token representation
    return cls_embedding.squeeze().cpu().detach().numpy()

In [8]:
# 태그 임베딩 생성
tag_embeddings = {}
for tag, sentences in tqdm(tag_examples.items(), desc="태그 임베딩 생성"):
    embeddings = np.vstack([get_kcbert_embedding(sentence) for sentence in sentences])
    tag_embeddings[tag] = embeddings.mean(axis=0)


태그 임베딩 생성: 100%|██████████| 7/7 [00:47<00:00,  6.83s/it]


In [9]:
# 리뷰 텍스트를 문장 단위로 분리하여 문장별 임베딩 생성
def get_sentence_embeddings(text):
    sentences = split_sentences(text)  # 리뷰를 문장 단위로 분리
    sentence_embeddings = [get_kcbert_embedding(sentence) for sentence in sentences]
    return sentence_embeddings


In [ ]:
# 리뷰 데이터의 모든 문장에 대해 개별 임베딩 생성
sentence_embeddings_per_review = [
    get_sentence_embeddings(text) for text in tqdm(filtered_data["review_after"], desc="리뷰 문장별 임베딩 생성")
]


리뷰 문장별 임베딩 생성:   0%|          | 0/8792 [00:00<?, ?it/s][Kss]: Because there's no supported C++ morpheme analyzer, Kss will take pecab as a backend. :D
For your information, Kss also supports mecab backend.
We recommend you to install mecab or konlpy.tag.Mecab for faster execution of Kss.
Please refer to following web sites for details:
- mecab: https://cleancode-ws.tistory.com/97
- konlpy.tag.Mecab: https://uwgdqo.tistory.com/363

c:\python-capstone\tag\Lib\site-packages\pecab\_tokenizer.py:265: RuntimeWarning: overflow encountered in scalar add
  from_pos_data.costs[idx]
리뷰 문장별 임베딩 생성:   0%|          | 5/8792 [00:20<11:41:12,  4.79s/it]c:\python-capstone\tag\Lib\site-packages\pecab\_tokenizer.py:274: RuntimeWarning: overflow encountered in scalar add
  least_cost += word_cost
리뷰 문장별 임베딩 생성:  44%|████▍     | 3878/8792 [1:06:56<3:20:05,  2.44s/it] 

In [ ]:
def get_review_tags(sentence_embeddings, tag_embeddings, review_text, threshold=0.6):
    tag_scores = Counter()

    # 각 문장 임베딩에 대해 태그 임베딩과 유사도 계산
    for sentence_embedding in sentence_embeddings:
        for tag, tag_embedding in tag_embeddings.items():
            sim = cosine_similarity([sentence_embedding], [tag_embedding]).item()
            weight = 2 if any(keyword in review_text for keyword in tag_keywords[tag]) else 1
            # 유사도에 가중치를 곱하여 태그 점수 증가
            if sim * weight > threshold:
                tag_scores[tag] += 1

    # 가장 높은 점수를 받은 상위 2개의 태그 선택
    top_2_tags = [tag for tag, count in tag_scores.most_common(2)]
    return top_2_tags

In [ ]:
# 각 리뷰의 문장별 임베딩을 사용해 태그를 할당하고 저장
assigned_tags_per_review = [
    get_review_tags(sentence_embeddings, tag_embeddings, review_text, threshold=0.6)
    for sentence_embeddings, review_text in tqdm(
        zip(sentence_embeddings_per_review, filtered_data["review_after"]), desc="태그 할당"
    )
]

In [ ]:
filtered_data["assigned_tags"] = assigned_tags_per_review

In [ ]:
# 각 리뷰의 태그 개수를 카운트하여 저장
def count_tags_per_lodging(filtered_data):
    tag_summary = pd.DataFrame(columns=["lodging_id", "tag", "count"])  # 태그 개수를 저장할 빈 DataFrame 생성

    # 각 숙소에 대해 태그 개수 집계
    for lodging_id, group in filtered_data.groupby("lodging_id"):
        tag_counts = Counter()  # 태그 개수를 세기 위한 카운터 생성
        for assigned_tags in group["assigned_tags"]:
            for tag in assigned_tags:
                tag_counts[tag] += 1  # 태그 개수 증가

        # 집계된 태그를 DataFrame으로 변환
        for tag, count in tag_counts.items():
            tag_summary = tag_summary.append({"lodging_id": lodging_id, "tag": tag, "count": count}, ignore_index=True)

    return tag_summary  # 집계된 태그 DataFrame 반환


In [ ]:
# 각 숙소의 태그 개수를 계산하고 새로운 DataFrame에 저장
tag_summary_df = count_tags_per_lodging(filtered_data)

# 원래 DataFrame에 태그 개수를 병합
filtered_data = filtered_data.merge(tag_summary_df, on="lodging_id", how="left", suffixes=("", "_summary"))


In [ ]:
# Step 10: 각 숙소에서 가장 많이 나온 태그 추가
def most_common_tag_per_lodging(tag_summary_df):
    common_tags = tag_summary_df.loc[tag_summary_df.groupby("lodging_id")["count"].idxmax()]  # 각 숙소에서 가장 높은 카운트의 태그 선택
    common_tags = common_tags[["lodging_id", "tag"]]  # 필요한 컬럼만 선택
    common_tags.columns = ["lodging_id", "most_common_tag"]  # 컬럼명 변경
    return common_tags  # 결과 반환

In [ ]:
# 가장 많이 나온 태그를 추가
most_common_tags_df = most_common_tag_per_lodging(tag_summary_df)
filtered_data = filtered_data.merge

In [ ]:
filtered_data

In [ ]:
filtered_data.to_csv("review_tags(kcbert_large, weight, final).csv", encoding='utf-8-sig', index=False)